# MNIST

## Import required packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import warnings

In [2]:
warnings.simplefilter('ignore')

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


## Outlining the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 50

In [4]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [5]:
warnings.simplefilter('ignore')

# this is included as the tensorflow installed is v2 and the code is written in v1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
# defining the input and output data
inputs = tf.placeholder(tf.float32,[None, input_size])
targets = tf.placeholder(tf.float32,[None, output_size])

# hidden layer 1
weights_1 = tf.get_variable("weights_1",[input_size,hidden_layer_size])
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])

output_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1)

# hidden layer 2
weights_2 = tf.get_variable("weights_2",[hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])

output_2 = tf.nn.relu(tf.matmul(output_1,weights_2) + biases_2)

#output layer
weights_3 = tf.get_variable("weights_3",[hidden_layer_size,output_size])
biases_3 = tf.get_variable("biases_3",[output_size])

outputs = tf.matmul(output_2,weights_3) + biases_3

In [7]:
# loss for our neural network
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)
mean_loss = tf.reduce_mean(loss)

#optimization function
optimize = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(mean_loss)

out_equal_targets = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

accuracy = tf.reduce_mean(tf.cast(out_equal_targets, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
# defining session
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

In [9]:
# dividing the training set in batches 
batch_size = 100
batches_number = mnist.train._num_examples // batch_size

# declaring the epoch size
max_epochs = 15

# declaring to check for overfitting in the loop
prev_validation_loss = 9999999

In [10]:
# training the dataset
for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0
    # loop for batches in each epoch
    for batch_counter in range(batches_number):
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        _, batch_loss = sess.run([optimize, mean_loss], feed_dict = {inputs: input_batch, targets: target_batch})
        curr_epoch_loss += batch_loss
        
    curr_epoch_loss /= batches_number
    
    # comparing the training with the validation set
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                    feed_dict = {inputs: input_batch, targets: target_batch})
    
    print('Epoch: ', (epoch_counter + 1),
         '. Training loss: ', '{0:.3f}'.format(curr_epoch_loss),
         '. Validation loss: ', '{0:.3f}'.format(validation_loss), 
         '. Validation accuracy:', '{0:.2f}'.format(validation_accuracy * 100.), '%')
    
    # checking for overfitting -> as validation loss increases
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss = validation_loss
    
print('End of training')

Epoch:  1 . Training loss:  0.410 . Validation loss:  0.209 . Validation accuracy: 94.10 %
Epoch:  2 . Training loss:  0.185 . Validation loss:  0.149 . Validation accuracy: 95.86 %
Epoch:  3 . Training loss:  0.138 . Validation loss:  0.127 . Validation accuracy: 96.40 %
Epoch:  4 . Training loss:  0.113 . Validation loss:  0.111 . Validation accuracy: 96.70 %
Epoch:  5 . Training loss:  0.095 . Validation loss:  0.104 . Validation accuracy: 96.92 %
Epoch:  6 . Training loss:  0.082 . Validation loss:  0.096 . Validation accuracy: 97.08 %
Epoch:  7 . Training loss:  0.072 . Validation loss:  0.094 . Validation accuracy: 97.28 %
Epoch:  8 . Training loss:  0.062 . Validation loss:  0.089 . Validation accuracy: 97.32 %
Epoch:  9 . Training loss:  0.055 . Validation loss:  0.095 . Validation accuracy: 97.42 %
End of training


## Testing

In [11]:
input_batch, target_batch = mnist.validation.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy], feed_dict = {inputs: input_batch, targets: target_batch})

test_accuracy_percent = test_accuracy[0] * 100

print('Test accuracy: ', '{0:,.2f}'.format(test_accuracy_percent), '%')

Test accuracy:  97.42 %
